In [ ]:
import requests as req
import os, json
from bs4 import BeautifulSoup as bs

# 隨機取得 User-Agent
'''
# 從外部資料來取得清單，清單預設儲存路徑: /tmp
ua = UserAgent(use_external_data=True)
# 從外部資料來取得清單，儲存在指定路徑
ua = UserAgent(use_external_data=True, cache_path=/home/fake_useragent.json)

更詳細的說明，請見以下網頁:
https://pypi.org/project/fake-useragent/
'''
from fake_useragent import UserAgent
ua = UserAgent(use_external_data=True)

# 建立儲存圖片的資料夾，不存在就新增
folderPath = 'line_stickers'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)
    
#放貼圖資訊用
listLineStickers = []

# 自訂標頭
my_headers = {
    'user-agent': ua.random
}

# 官方 LINE 貼圖的網址
url = 'https://store.line.me/stickershop/product/19716012/zh-Hant'

# 將自訂標頭加入 GET 請求中
res = req.get(url, headers = my_headers)

# 建立 soup 物件
soup = bs(res.text, 'lxml')

'''
備註:
1. soup.select()：回傳的結果是元素集合（list 型態，BeautifulSoup ResultSet）
2. soup.select_one()：回傳的結果是單一元素（BeautifulSoup Result）
'''
# 取得放置貼圖的 li 元素 (list 型態)
li_elements = soup.select("ul.mdCMN09Ul.FnStickerList > li.mdCMN09Li.FnStickerPreviewItem")


# 逐一取得 li 元素中的 data-preview 資訊
for li in li_elements:
    # 取得 data-preview 屬性的值(字串)
    strJson = li['data-preview'] # 另一種寫法：li.get("data-preview")
    
    #把屬性的值(字串)轉成物件 
    obj = json.loads(strJson)
    
    # 將重要資訊放置在 list 當中，幫助我們稍候進行資料下載與儲存
    listLineStickers.append(obj)

# 下載圖片
for obj in listLineStickers: 
    os.system(f"curl {obj['staticUrl']} -o {folderPath}/{obj['id']}.png")
    print(f"貼圖ID: {obj['id']}, 下載連結: {obj['staticUrl']}")